In [1]:
import numpy as np
import time


from numpy.random import random
from collections import Counter

# Synthesis

I present here a solution using a dynamic programming approach, ie breaking down the problem in smaller sub-problems from which I extract the solution to solve my initial problem. This notebook contains a detailed explanation of my thought process and experimentations. A functionnal version of the code is also in the 'src' folder, wrapped into a class. The required documentation is in the 'README.md' at the root of this repository.

Here, the problem is a 2d version of the Knapsack problem:

>Given a box with size (W, H) and a collection of N rectangles with sizes (w_i, h_i) and values v_i > 0, find an arrangement of non-overlapping rectangles that fits within the box and maximizes their total value. Rectangles may be used more than once. Rectangles may be rotated. N may be very large.

The instructions say that the rectangles may be rotated. We build our optimized solution for rectangles that may be rotated of __90 degree__ only. Other rotations won't be optimal as we fill a bigger rectangle, there will be necessarily wasted space.

The next cell contains the test we will use. It needs to be run.

In [2]:
def timeit(f):

    def timed(*args, **kw):

        ts = time.time()
        result = f(*args, **kw)
        te = time.time()

        print 'func:%r args:[%r, %r] took: %2.4f sec' % \
          (f.__name__, args, kw, te-ts)
        return result

    return timed

@timeit
def test_simple(function):
    setup = (45, 48, [(4, 28, 4.0), (8, 10, 20.0)])
    F, a = function(*setup)
    assert(F[-1, -1] == 520)
    assert(len(a[setup[0], setup[1]]) == 26)

# Benchmark: large number of rectangles with many including better ones
def including_sample(w0=1, h0=1, v0=5, Nx=10, Ny=10):
    '''
    Build Nx*Ny rectangles with w0< w <= w0 + N and h0< h <= h0 + N which contain the
    initial rectanle (w0, h0) but with a lower value.
    '''
    rectangles = [(w0, h0, v0)]
    for i in xrange(1, Nx+1):
        for j in xrange(1, Ny+1):
            v = v0*random()
            r = (w0 + i, h0 + j, v)
            rectangles.append(r)
    return rectangles

@timeit
def test_including_sample(function):
    # Define global setup
    W = 45
    H = 48
    ref0 = (3, 4, 2.)
    ref1 = (8, 10, 25.0)
    ref2 = (7, 9, 14)
    ref3 = (17, 14, 80)
    rectangles = [ref1, ref2, ref3]
    
    # Generate worse rectangles
    rectangles += including_sample(*ref0, Nx=40, Ny=40)
    rectangles += including_sample(*ref1, Nx=35, Ny=35)
    rectangles += including_sample(*ref2, Nx=35, Ny=35)
    rectangles += including_sample(*ref3, Nx=25, Ny=30)
    
    # Run experiment
    setup = (W, H, rectangles)
    F, a = function(*setup)
    assert(F[-1, -1] == 695.)
    assert(len(a[setup[0], setup[1]]) == 17)

@timeit
def test_small_rectangles(function):
    # Define global setup
    W = 45
    H = 48
    ref0 = (1, 1, 0.1)
    ref1 = (8, 10, 25.0)
    ref2 = (7, 9, 14)
    ref3 = (17, 14, 80)
    rectangles = [ref1, ref2, ref3]

    # Generate worse rectangles
    rectangles += including_sample(*ref0, Nx=40, Ny=40)
    rectangles += including_sample(*ref1, Nx=35, Ny=35)
    rectangles += including_sample(*ref2, Nx=35, Ny=35)
    rectangles += including_sample(*ref3, Nx=25, Ny=30)
    
    # Run experiment
    setup = (W, H, rectangles)
    F, a = function(*setup)
    # Float comparison
    assert(np.abs(F[-1, -1] - 694.4) < 10**(-8))
    assert(len(a[setup[0], setup[1]]) == 106)
    

# Script to run test
def test_main(function):
    test_simple(function)
    print('1')
    test_including_sample(function)
    print('2')
    test_small_rectangles(function)
    print('3')


### Dynamic Solution

The idea is to compute the optimal arrangement for all the rectangles (w,h) with 1<= w <= W and 1 <= h <= H, starting from the smallest and finish with the objective, ie (W, H). Let's store each of this sub-solution into a matrix F and a dictionnary arrangement:

* __F[i, j]__ : stores the value of the optimal arrangement in the rectangle of size (i, j)
* __arrangement[(i, j)]__: stores the list of rectangles contained in the optimal arrangement in the rectangle of size (i, j) in the following format : [(x1, y1, x2, y2, v)]

Now we can decribe the different steps of the algorithm:

* Initialization of F at zeros (and arrangement empty)
* For each sub-rectangle, ie in our notation for each (i, j) with 1<= i <= W and 1 <= j <= H we set F[i,j] and arrangement[(i,j)] to the larger rectangle which fits in it, if one exists.
* Construction of the optimal solution starting from the bottom. Iteration on (i,j) to build F[i,j] and arrangement[(i, j)] with previously computed values. Here is the pseudo code:


    for i = 1, W
        for j = 1, H
            # along axis x
            for k = 1, [i/2]
                value_merged = F[k, j] + F[i-k,j]
                if value_merged > F[i, j]
                     F[i, j] = value_merged
                     Updating arrangement[(i,j)] with the concatenation of the two arrangements (if not empty)         
            # along axis y
            for k = 1, [j/2]
                value_merged = F[i, k] + F[i,j-k]
                if value_merged > F[i, j]
                     F[i, j] = value_merged
                     Updating arrangement[(i,j)] with the concatenation of the two arrangements (if not empty)
                     
At the end of the execution, we can retrieve the solution for the tuple $(i, j) = (W, H)$ inside F and arrangement.

__Complexity__:


* step 1 : in time $O (WH) $
* step 2 : in time $ O( WH \times N) $ (naive linear scan of each rectangle for each cell of F)
* step 3: in time $ O (WH \times (W + H))$ (each final subloop along the two axis induce a time complexity of the size of the axis on average)
* Memory: $ O( WH \times |R|)$ ( $O(WH)$ for F and the dictionnary arrangement stores in the worst case $O ( WH \times \alpha(W, H, N))$, with $\alpha(W, H)$ the maximum number of rectangles we can fit inside the (W, H) rectangle.)

NB: a brute force algorithm will build all the possible arrangements and keep the best one. First, it picks k rectangles among N to fill (W,H) (choice among $2^N$ possibilities, some may be equivalent to other for the rectangle too big) and tries all of the combinaisons in the target space. This is at least $O(WH \times N_rec)$ with N_rec the number of rectangle that can fit inside the target area (this number may be larger than N and depends on W, H, N. In total we have a complexity already higer than $O(WHN \times 2^N)$

We provide a functionnal and detailed version of the algorithm: (the final version in 'src' is less detailed and less redundant), I provide here the raw one to make the full algorithm clearer:

In [3]:
def dynamic_solution(W, H, rectangles):
    # Store the total value
    F = np.zeros((W+1, H+1))
    # Stores the arrangement for each configuration (i,j)
    # key: (i,j) value: [(x1, y1, x2, y2, v)]
    arrangement = {}
    
    # Best configuration with 1 rectangle
    for r in rectangles:
        w, h, v = r
        for i in xrange(1, W+1):
            for j in xrange(1, H+1):
                # Rotations of 90° possible
                if ((w <= i and h <= j) or (h <= i and w <= j)) and (v > F[i, j]):
                    F[i, j] = v
                    if not (w <= i and h <= j):
                        arrangement[(i,j)] = [(0, 0, h, w, v)]
                    else: # by default if two positions possible
                        arrangement[(i,j)] = [(0, 0, w, h, v)]
    # Iteration
    for i in xrange(1,W+1):
        for j in xrange(1,H+1):
            # Combining 2 sub configurations
            # Along x
            for k_x in xrange(1,i/2+1):
                v_local = F[k_x, j] + F[i - k_x, j]
                if v_local > F[i, j]:
                    F[i,j] = v_local
                    # Updating arrangement
                    if (k_x, j) not in arrangement:
                        arrangement[(i, j)] = arrangement[(i - k_x, j)]
                    elif (i - k_x, j) not in arrangement:
                        arrangement[(i, j)] = arrangement[(k_x, j)]
                    else:
                        # concatenating arrangement with index shift along x for the second one
                        a_shifted = [(e[0] + i - k_x, e[1], e[2] + i - k_x, e[3], e[4]) for e in arrangement[(k_x, j)]]
                        arrangement[(i, j)] = arrangement[(i - k_x, j)] + a_shifted
                    # Updating arrangement
                    
                
            # Along y
            for k_y in xrange(1,j/2+1):
                merged_value = F[i, k_y] + F[i, j - k_y]
                if merged_value > F[i, j]:
                    F[i,j] = merged_value
                    # Updating arrangement
                    if (i, k_y) not in arrangement:
                        arrangement[(i, j)] = arrangement[(i, j - k_y)]
                    elif (i, j - k_y) not in arrangement:
                        arrangement[(i, j)] = arrangement[(i, k_y)]
                    else:
                        # concatenating arrangement with index shift along y for the second one
                        a_shifted = [(e[0], e[1] + j - k_y, e[2], e[3] + j - k_y, e[4]) for e in arrangement[(i, k_y)]]
                        arrangement[(i, j)] = arrangement[(i, j - k_y)] + a_shifted

    return F, arrangement

In [4]:
%%time
test_main(dynamic_solution)

func:'test_simple' args:[(<function dynamic_solution at 0x1055d3500>,), {}] took: 0.0490 sec
1
func:'test_including_sample' args:[(<function dynamic_solution at 0x1055d3500>,), {}] took: 1.5405 sec
2
func:'test_small_rectangles' args:[(<function dynamic_solution at 0x1055d3500>,), {}] took: 1.6396 sec
3
CPU times: user 3.19 s, sys: 21.5 ms, total: 3.22 s
Wall time: 3.23 s


### Improvements

### Arrangement storage

The idea is here to decrease the memory storage. The arrangement dictionnary needs a lot of space if we can fit a high number of rectangles in the target area. One idea is to change the arrangement format and just care about the number of each kind of rectangles in it. Here the memory complexity becomes bounded, ie $O(WH \times N)$ for the new dictionnary called __configuration__ which contains Python Counters.

In [5]:
def dynamic_solution_low_memory(W, H, rectangles):
    # Store the total value
    F = np.zeros((W+1, H+1))
    # Store the rectangles present in each configuration
    # using a mapping rectangle2index
    rectangles2indexes = {r:i for i, r in enumerate(rectangles)}
    indexes2rectangles = {i:r for i, r in enumerate(rectangles)}
    configuration = {}
    
    # Best configuration with 1 rectangle
    for r in rectangles:
        w, h, v = r
        for i in xrange(1, W+1):
            for j in xrange(1, H+1):
                # Rotations of 90° possible
                if ((w <= i and h <= j) or (h <= i and w <= j)) and (v > F[i, j]):
                    F[i, j] = v
                    configuration[(i,j)] = Counter([rectangles2indexes[r]])
    # Iteration
    for i in xrange(1,W+1):
        for j in xrange(1,H+1):
            # Combining 2 sub configurations
            # Along x
            for k_x in xrange(1,i/2+1):
                v_local = F[k_x, j] + F[i - k_x, j]
                if v_local > F[i, j]:
                    F[i,j] = v_local
                    # Updating configuration
                    if (k_x, j) not in configuration:
                        configuration[(i, j)] = configuration[(i - k_x, j)]
                    elif (i - k_x, j) not in configuration:
                        configuration[(i, j)] = configuration[(k_x, j)]
                    else:
                        c = Counter(configuration[(k_x, j)])
                        c.update(configuration[(i - k_x, j)])
                        configuration[(i, j)] = c
                
            # Along y
            for k_y in xrange(1,j/2+1):
                v_local = F[i, k_y] + F[i, j - k_y]
                if v_local > F[i, j]:
                    F[i,j] = v_local
                    # Updating configuration
                    if (i, k_y) not in configuration:
                        configuration[(i, j)] = configuration[(i, j - k_y)]
                    elif (i, j - k_y) not in configuration:
                        configuration[(i, j)] = configuration[(i, k_y)]
                    else:
                        c = Counter(configuration[(i, k_y)])
                        c.update(configuration[(i, j - k_y)])
                        configuration[(i, j)] = c

    return F, configuration, indexes2rectangles

In [6]:
%%time
# Testing

# Define global setup
W = 45
H = 48
ref1 = (8, 10, 25.0)
ref2 = (7, 9, 14)
ref3 = (17, 14, 80)
rectangles = [ref1, ref2, ref3]

setup = (W, H, rectangles)

F, c, m = dynamic_solution_low_memory(*setup)

CPU times: user 68.8 ms, sys: 5.21 ms, total: 74.1 ms
Wall time: 71.2 ms


In [7]:
# Printing
print F[W, H]
print c[(W, H)]
print m

688.0
Counter({2: 7, 0: 4, 1: 2})
{0: (8, 10, 25.0), 1: (7, 9, 14), 2: (17, 14, 80)}


### Time complexity: Initialization step

We can improve the initialization step where we fill F and arrangement with the raw value of the rectangle with the largest value that fits in it, if exists. We could sort the rectangles by their values and consider them in decreasing order of the values. This will give a new time complexity of $O(N log(N))$ for this loop instead of $O(WH \times N)$.

The idea is to fill from the upper right corner (W, H) to the lower left one F and arrangement. First, we consider the rectangle with the largest value and fill all the cells (i,j) if the subrectangle (i,j) contains this rectangle. Then we consider the next top value rectangle and fill new cells if it's smaller in at least one dimension and so on. 

The tricky part is the update of F and arrangement for a new top value rectangle. We have three different cases summarized by the figures. We go here from the rectangle with the highest value (in blue) to the next one (in red). The ligt rectangle represent the real one and the dark one the cells of F that need to be set to the value of the corresponding rectangle. For each case we have different number of subareas for the area with red cells to set (resp. 2, 3 and 4). These are exactly those leading to the updates statements in the code

<img width=400 height=200 src="cas1.png"/> 

<img width=400 height=200 src="cas2.png"/> 

<img width=400 height=200 src="cas3.png"/> 


The new code is much faster than the previous one on the unit-tests (especially one with a high number of rectangles):


In [8]:
def update_in_range(mydict, updates):
    '''
    Update in place my dict in the range defined in udpates.
    args:
        updates: dictionnary (imin, imax, jmin, jmax) : value
    '''
    for ranges, value in updates.iteritems():
        for i in xrange(ranges[0], ranges[1]):
            for j in xrange(ranges[2], ranges[3]):
                mydict[(i, j)] = value

In [9]:
def dynamic_solution_optim_init(W, H, rectangles):
    N = len(rectangles)
    # Store the total value
    F = np.zeros((W+1, H+1))
    # Stores the arrangement for each configuratin (i,j)
    # key: (i,j) value: [(x1, y1, x2, y2, v)]
    arrangement = {}
    
    # Best configuration with 1 rectangle
    # Sorting the rectangle list by decreasing values
    # Complexity is O(N log(N))
    rectangles.sort(reverse=True, key= lambda x:x[2])
    ind_max = max(W, H) + 1
    ind_min = min(W, H) + 1
    curr_rect = 0
    while ind_max > 0 and ind_min > 0 and curr_rect < N:
        r = rectangles[curr_rect]
        w, h, v = r
        min_d = min(w, h)
        max_d = max(w, h)
        if max_d < ind_max or min_d < ind_min:
            # Case 1: 2 portions for the enxt rectangle
            if min_d >= ind_min:
                # Setting F
                F[min_d:max_d, max_d:ind_max] = v
                F[max_d:ind_max, min_d:ind_max] = v
                # Setting arragnement
                updates = {(min_d, max_d, max_d, ind_max): [(0, 0, min_d, max_d, v)],
                          (max_d, ind_max, min_d, ind_max): [(0, 0, max_d, min_d, v)]}
                update_in_range(arrangement, updates)
            # Case 2: 3 portions for the enxt rectangle
            elif max_d < ind_min:
                # Setting F
                F[min_d:ind_min, max_d:H+1] = v
                F[ind_min:ind_max, ind_min:ind_max] = v
                F[max_d:W+1, min_d:ind_min] = v
                # Setting arrangement
                updates = {(min_d, ind_min, max_d, H+1): [(0, 0, min_d, max_d, v)],
                           (ind_min, ind_max, ind_min, ind_max): [(0, 0, min_d, max_d, v)],
                          (max_d, W+1, min_d, ind_min): [(0, 0, max_d, min_d, v)]}
                update_in_range(arrangement, updates)
            # Case 3: 4 portions for the enxt rectangle
            else:
                # Setting F
                F[min_d:ind_min, max_d:H+1] = v
                F[max_d:W+1, min_d:ind_min] = v
                F[min_d:max_d, max_d:ind_max] = v
                F[max_d:ind_max, min_d:ind_max] = v
    
                # Setting arrangement
                updates = {(min_d, ind_min, max_d, H+1): [(0, 0, min_d, max_d, v)],
                           (min_d, max_d, max_d, ind_max): [(0, 0, min_d, max_d, v)],
                           (max_d, ind_max, min_d, ind_max): [(0, 0, max_d, min_d, v)],
                           (max_d, W+1, min_d, ind_min): [(0, 0, max_d, min_d, v)]}
                update_in_range(arrangement, updates)

            ind_min = min_d
            ind_max = max_d
        curr_rect += 1
    
    # Iteration
    for i in xrange(1,W+1):
        for j in xrange(1,H+1):
            # Combining 2 sub configurations
            # Along x
            for k_x in xrange(1,i/2+1):
                v_local = F[k_x, j] + F[i - k_x, j]
                if v_local > F[i, j]:
                    F[i,j] = v_local
                    # Updating arrangement
                    if (k_x, j) not in arrangement:
                        arrangement[(i, j)] = arrangement[(i - k_x, j)]
                    elif (i - k_x, j) not in arrangement:
                        arrangement[(i, j)] = arrangement[(k_x, j)]
                    else:
                        # concatenating arrangement with index shift along x for the second one
                        a_shifted = [(e[0] + i - k_x, e[1], e[2] + i - k_x, e[3], e[4]) for e in arrangement[(k_x, j)]]
                        arrangement[(i, j)] = arrangement[(i - k_x, j)] + a_shifted
                    # Updating arrangement
                    
                
            # Along y
            for k_y in xrange(1,j/2+1):
                merged_value = F[i, k_y] + F[i, j - k_y]
                if merged_value > F[i, j]:
                    F[i,j] = merged_value
                    # Updating arrangement
                    if (i, k_y) not in arrangement:
                        arrangement[(i, j)] = arrangement[(i, j - k_y)]
                    elif (i, j - k_y) not in arrangement:
                        arrangement[(i, j)] = arrangement[(i, k_y)]
                    else:
                        # concatenating arrangement with index shift along y for the second one
                        a_shifted = [(e[0], e[1] + j - k_y, e[2], e[3] + j - k_y, e[4]) for e in arrangement[(i, k_y)]]
                        arrangement[(i, j)] = arrangement[(i, j - k_y)] + a_shifted

    return F, arrangement

In [10]:
%%time
test_main(dynamic_solution_optim_init)

func:'test_simple' args:[(<function dynamic_solution_optim_init at 0x1055be938>,), {}] took: 0.0507 sec
1
func:'test_including_sample' args:[(<function dynamic_solution_optim_init at 0x1055be938>,), {}] took: 0.0560 sec
2
func:'test_small_rectangles' args:[(<function dynamic_solution_optim_init at 0x1055be938>,), {}] took: 0.1040 sec
3
CPU times: user 201 ms, sys: 10.8 ms, total: 212 ms
Wall time: 212 ms


The same unittests are executed in __213 ms__ compared to __3.28s__ for the previous version

### Symmetrie

As we can rotate from 90 degree each rectangle (ie flip i and j) I have the feeling that this problem has a lot of symmetrie. As a result, I tried to optimize the main loop of the step 3 of the algorithm. For example we could store F as a "triangular" matrix (actually F is not square but we could have a triangular version for the submatrix F(min(W,H), min(W,H)). This will divide by 2 the iterations. This is not as easy as expected as the updates along the axis y and x in the inner loop may use cells from both side, so we have to include many if statements to be sure to stay in the right triangular part. 

The following figure illustrates this:

<img width=400 height=200 src="sym_along_x.png"/> 

In that figure, the point (k,j) and (i-k, j) do not belong to the same triangular matrice. We could flip the first one with (j,k) which belongs to the same triangular matrice but we need to add many if statements dor these flips.


This optimization is still pending as its positive impact is not obvious. It may make the codes way more complicated for only a small gain.

# Future Work

The deliverable is a working solution of this 2d knapsack problem. I successfully optimized the initialization of the algorithm decreasing by a factor 10 the running time. I wrapped in in a class with unit test and some useful function to produce an output but also to draw the built arrangement (very useful to debugg).

However, there is still room for improvement:

As suggested by the problem statement, we could extend this algorithm for non-float rectangle dimension. The main issue is here that the set of different stages in the step 3 of the algorithm (one stage is combining the solution of two subproblems to solve a bigger one) is now much larger. This is due to the concatenation of rectangles which will form only on marginal case a rectangle with integer dimension. The questions raised here are: How to process these new multiple stages? How to increase the dimension of F as the integer one may not be relevant anymore?

We could also improve the memory storage. For example, the arrangement dictionary actually contains the information of the matrix F (we just need to sum the values of each rectangle for one arrangement to get the corresponding F value). Both are kept for computation time optimization, but it's the memory is an issue we can remove it. Also, we could look for a reconstruction algorithm given the F matrix. This is a classical approach in dynamic programming: given the set of subproblems soved, we can construct the path leading to the global solution. We won't need to store the arrangement dictionnary if this is possible (or maybe a lower version).

Lastly, we could improve the initialization step while filtering out some rectangles in the rectangle list of we know they are worst than another one (ie smaller value for both dimensions bigger). 